<a href="https://colab.research.google.com/github/DanielMerkdo/analitica_marketing/blob/main/c_modelado_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CARGUE Y LECTURA TABLA FINAL

In [42]:
# Montar drive y crear path para sys y para directorio de trabajo

from google.colab import drive
drive.mount('/content/drive')

import sys
import os

# Define el path al directorio de trabajo actual
path = "/content/drive/My Drive/cod/analitica_marketing"


sys.path.append(path) ### para importar archivo de funciones propias a través de import

os.chdir(path) ### para que por defecto suba y descargue archivos partiendo de esa ruta

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
 ### Importar librerias
 import numpy as np
 import pandas as pd
 import sqlite3 as sql
 from sklearn.preprocessing import MinMaxScaler
 from ipywidgets import interact
 from sklearn import neighbors
 import joblib

In [44]:
import a_funciones as funciones     # Importar las funciones
import importlib
importlib.reload(funciones)

<module 'a_funciones' from '/content/drive/My Drive/cod/analitica_marketing/a_funciones.py'>

In [45]:
 ### Conectarse con la base de datos
 conex= sql.connect('data/db_movies_final.db')
 cur= conex.cursor()

In [46]:
 ### Ver las tablas que hay en la base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('db_movies_final',)]

In [47]:
column_names_query = "SELECT name FROM PRAGMA_TABLE_INFO('db_movies_final')"
columns_ratings = pd.read_sql(column_names_query, conex)
columns_ratings


,name
0,user_id
1,movie_Id
2,movie_rating
3,rating_year
4,movie_title
5,movie_year
6,Action
7,Adventure
8,Animation
9,Children


In [48]:
final_movies= pd.read_sql("""SELECT * FROM db_movies_final""", conex)
final_movies

,user_id,movie_Id,movie_rating,rating_year,movie_title,movie_year,Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,2000,Toy Story,1995,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,2000,Grumpier Old Men,1995,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,2000,Heat,1995,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,2000,Seven (a.k.a. Se7en),1995,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,2000,"Usual Suspects, The",1995,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55999,609,828,3.0,1996,"Adventures of Pinocchio, The",1996,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
56000,609,833,3.0,1996,High School High,1996,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56001,609,892,3.0,1996,Twelfth Night,1996,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
56002,609,1056,3.0,1996,Jude,1996,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#SISTEMAS BASADOS EN POPULARIDAD



In [ ]:
### 10 Mejores peliculas calificadas que no tengan nunca una calificacion de 0.0 y año de la pelicula mayor a 1990, y que hayan sido rankeadas mas de 100 veces
pd.read_sql("""select movie_title, movie_year,
            avg(movie_rating) as avg_rat,
            count(*) as count_rating
            from db_movies_final
            where movie_rating<>0.0
             AND movie_year > 1990
            group by movie_title
            having count_rating >100
            order by avg_rat desc
            limit 10
            """, conex)

,movie_title,movie_year,avg_rat,count_rating
0,"Shawshank Redemption, The",1994,4.439286,280
1,Fight Club,1999,4.252778,180
2,"Usual Suspects, The",1995,4.251462,171
3,"Dark Knight, The",2008,4.235537,121
4,Schindler's List,1993,4.207895,190
5,"Matrix, The",1999,4.200000,235
6,Forrest Gump,1994,4.200000,285
7,"Silence of the Lambs, The",1991,4.156379,243
8,Pulp Fiction,1994,4.152985,268
9,Fargo,1996,4.151724,145


In [ ]:
### La peliculas mas vistas con promedio
pd.read_sql("""
    SELECT movie_title,movie_year,
           AVG(CASE WHEN movie_rating = 0 THEN NULL ELSE movie_rating END) AS avg_cal,
           COUNT(*) AS count_rating
    FROM db_movies_final
    GROUP BY movie_title
    ORDER BY count_rating DESC
    LIMIT 10
""", conex)

,movie_title,movie_year,avg_cal,count_rating
0,Forrest Gump,1994,4.200000,285
1,"Shawshank Redemption, The",1994,4.439286,280
2,Pulp Fiction,1994,4.152985,268
3,"Silence of the Lambs, The",1991,4.156379,243
4,"Matrix, The",1999,4.200000,235
5,Star Wars: Episode IV - A New Hope,1977,4.210900,211
6,Braveheart,1995,4.060976,205
7,Jurassic Park,1993,3.769036,197
8,Schindler's List,1993,4.207895,190
9,Terminator 2: Judgment Day,1991,3.954787,188


#SISTEMA DE RECOMENDACIÓN BASADO EN CONTENIDO DE UN PRODUCTO

In [ ]:
final_movies2= pd.read_sql("""select * from db_movies_final""", conex)
final_movies2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56004 entries, 0 to 56003
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       56004 non-null  int64  
 1   movie_Id      56004 non-null  int64  
 2   movie_rating  56004 non-null  float64
 3   rating_year   56004 non-null  int64  
 4   movie_title   56004 non-null  object 
 5   movie_year    56004 non-null  int64  
 6   Action        56004 non-null  int64  
 7   Adventure     56004 non-null  int64  
 8   Animation     56004 non-null  int64  
 9   Children      56004 non-null  int64  
 10  Comedy        56004 non-null  int64  
 11  Crime         56004 non-null  int64  
 12  Documentary   56004 non-null  int64  
 13  Drama         56004 non-null  int64  
 14  Fantasy       56004 non-null  int64  
 15  Film-Noir     56004 non-null  int64  
 16  Horror        56004 non-null  int64  
 17  IMAX          56004 non-null  int64  
 18  Musical       56004 non-nu

In [ ]:
# Se eliminan los registros duplicados, solo se requiere el catalogo de las peliculas
final_movies3 = final_movies2.drop_duplicates(subset='movie_Id', keep='first').reset_index(drop=True)


In [ ]:
##### escalar para que año esté en el mismo rango ###
sc=MinMaxScaler()
final_movies3[["movie_yearsc"]]=sc.fit_transform(final_movies3[['movie_year']])
final_movies3

,user_id,movie_Id,movie_rating,rating_year,movie_title,movie_year,Action,Adventure,Animation,Children,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_yearsc
0,1,1,4.0,2000,Toy Story,1995,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0.765306
1,1,3,4.0,2000,Grumpier Old Men,1995,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.765306
2,1,6,4.0,2000,Heat,1995,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0.765306
3,1,47,5.0,2000,Seven (a.k.a. Se7en),1995,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0.765306
4,1,50,5.0,2000,"Usual Suspects, The",1995,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0.765306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,586,34332,4.0,2018,Sky High,2005,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0.867347
3214,594,4748,4.5,2005,3 Ninjas,1992,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0.734694
3215,596,42004,3.5,2018,Transamerica,2005,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.867347
3216,597,2014,4.0,1999,Freaky Friday,1977,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.581633


In [ ]:
#eliminar columnas que no se requieren para el analisis
final_moviesknn=final_movies3.drop(columns=['user_id','movie_Id','movie_rating','movie_year','rating_year','movie_title'])


In [ ]:
final_moviesknn

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_yearsc
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.765306
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.765306
2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0.765306
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0.765306
4,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0.765306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.867347
3214,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.734694
3215,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.867347
3216,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.581633


In [ ]:
## el coseno de un angulo entre dos vectores es 1 cuando son perpendiculares y 0 cuando son paralelos(indicando que son muy similar324e-06	3.336112e-01	3.336665e-01	3.336665e-es)
model = neighbors.NearestNeighbors(n_neighbors=20, metric='cosine')
model.fit(final_moviesknn)
dist, idlist = model.kneighbors(final_moviesknn)

In [ ]:
distancias=pd.DataFrame(dist) ## devuelve un ranking de la distancias más cercanas para cada fila(pelicula)
id_list=pd.DataFrame(idlist) ## para saber esas distancias a que item corresponde

In [ ]:
####ejemplo para una pelicula
movies_list_name = []
movie_name='Forrest Gump'
movie_id = final_movies3[final_movies3['movie_title'] == movie_name].index ### extraer el indice de la pelicula
movie_id = movie_id[0] ## si encuentra varios solo guarde uno

for newid in idlist[movie_id]:
        movies_list_name.append(final_movies3.loc[newid].movie_title) ### agrega el nombre de cada una de los id recomendados

movies_list_name


['Forrest Gump',
 'Life Is Beautiful (La Vita è bella)',
 'It Could Happen to You',
 "Nobody's Fool",
 'Corrina, Corrina',
 'Postman, The (Postino, Il)',
 'Reality Bites',
 'Eat Drink Man Woman (Yin shi nan nu)',
 'American President, The',
 'Something to Talk About',
 'Mighty Aphrodite',
 'Don Juan DeMarco',
 'Waiting to Exhale',
 'Sleepless in Seattle',
 'Son in Law',
 'Emma',
 'Tin Cup',
 'English Patient, The',
 'Beautiful Girls',
 'Twelfth Night']

In [ ]:
####Se genera la lista de 10 peliculas mas cercanas
def MovieRecomender(movie_name= list(final_movies3["movie_title"].value_counts().index)):
    movie_list_name= []
    movie_id= final_movies3[final_movies3["movie_title"]== movie_name].index
    movie_id= movie_id[0]

    # Obtener solo los 10 IDs más cercanos (ignorando la película misma si es necesario)
    nearest_ids = idlist[movie_id][1:11]  # [1:11] salta el 0 (es la misma peli) y toma los 10 siguientes

    for newid in nearest_ids:
      movie_list_name.append(final_movies3.loc[newid].movie_title)
    return movie_list_name


In [ ]:
print(interact(MovieRecomender))

interactive(children=(Dropdown(description='movie_name', options=('King Kong', 'Misérables, Les', 'Avengers, T…

<function MovieRecomender at 0x7ac6c9d50ea0>


# SISTEMA DE RECOMENDACION BASADO EN CONTENIDO TODO LO VISTO POR EL USUARIO

In [ ]:
final_movies4= pd.read_sql("""select * from db_movies_final""", conex)
final_movies4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56004 entries, 0 to 56003
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       56004 non-null  int64  
 1   movie_Id      56004 non-null  int64  
 2   movie_rating  56004 non-null  float64
 3   rating_year   56004 non-null  int64  
 4   movie_title   56004 non-null  object 
 5   movie_year    56004 non-null  int64  
 6   Action        56004 non-null  int64  
 7   Adventure     56004 non-null  int64  
 8   Animation     56004 non-null  int64  
 9   Children      56004 non-null  int64  
 10  Comedy        56004 non-null  int64  
 11  Crime         56004 non-null  int64  
 12  Documentary   56004 non-null  int64  
 13  Drama         56004 non-null  int64  
 14  Fantasy       56004 non-null  int64  
 15  Film-Noir     56004 non-null  int64  
 16  Horror        56004 non-null  int64  
 17  IMAX          56004 non-null  int64  
 18  Musical       56004 non-nu

In [ ]:
final_movies5 = final_movies4.drop_duplicates(subset='movie_Id', keep='first').reset_index(drop=True)

In [ ]:
##### escalar para que año esté en el mismo rango ###
sc=MinMaxScaler()
final_movies5[["movie_yearsc"]]=sc.fit_transform(final_movies5[['movie_year']])
final_movies5

,user_id,movie_Id,movie_rating,rating_year,movie_title,movie_year,Action,Adventure,Animation,Children,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_yearsc
0,1,1,4.0,2000,Toy Story,1995,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0.765306
1,1,3,4.0,2000,Grumpier Old Men,1995,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.765306
2,1,6,4.0,2000,Heat,1995,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0.765306
3,1,47,5.0,2000,Seven (a.k.a. Se7en),1995,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0.765306
4,1,50,5.0,2000,"Usual Suspects, The",1995,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0.765306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,586,34332,4.0,2018,Sky High,2005,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0.867347
3214,594,4748,4.5,2005,3 Ninjas,1992,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0.734694
3215,596,42004,3.5,2018,Transamerica,2005,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.867347
3216,597,2014,4.0,1999,Freaky Friday,1977,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.581633


In [ ]:
#eliminar columnas que no se requieren para el analisis
final_moviesT1=final_movies5.drop(columns=['movie_year','rating_year'])

In [ ]:
#eliminar columnas que no se requieren para el analisis
final_moviesT2=final_movies5.drop(columns=['movie_year','rating_year'])
final_moviesT2

,user_id,movie_Id,movie_rating,movie_title,Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,Toy Story,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,Grumpier Old Men,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,Heat,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,Seven (a.k.a. Se7en),0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,"Usual Suspects, The",0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,586,34332,4.0,Sky High,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3214,594,4748,4.5,3 Ninjas,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3215,596,42004,3.5,Transamerica,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3216,597,2014,4.0,Freaky Friday,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
##Seleccionar usuarios para recomendaciones
usuarios=pd.read_sql('select distinct (user_id) as user_id from db_movies_final',conex)
usuarios

In [ ]:
user_id=312 ### para ejemplo manual


def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):

    ###seleccionar solo los ratings del usuario seleccionado
    ratings=pd.read_sql('select *from db_movies_final where user_id=:user',conex, params={'user':user_id})

    ###convertir ratings del usuario a array
    l_movies_r=ratings['movie_Id'].to_numpy()

    ###agregar la columna de movie_id y titulo de la pelicula  para filtrar y mostrar nombre
    final_moviesT1[['movie_Id','movie_title']]=final_moviesT2[['movie_Id','movie_title']]

    ### filtrar peliculas calificados por el usuario
    movies_r= final_moviesT1[final_moviesT1['movie_Id'].isin(l_movies_r)]

    ## eliminar columna nombre e isbn
    movies_r=movies_r.drop(columns=['movie_Id','movie_title'])
    movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    ##centroide o perfil del usuario
    centroide=movies_r.groupby("indice").mean()

    ### filtrar peliculas no vistas
    movies_nr=final_moviesT1[~final_moviesT1['movie_Id'].isin(l_movies_r)]
    ## eliminbar movie_id y nombre de la pelicula
    movies_nr=movies_nr.drop(columns=['movie_Id','movie_title'])


      ### entrenar modelo
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(movies_nr)
    dist, idlist = model.kneighbors(centroide)

    ids=idlist[0] ### queda en un array anidado, para sacarlo
    recomend_b=final_moviesT2.loc[ids][['movie_Id', 'movie_title']]
    vistas=final_moviesT2[final_moviesT2['movie_Id'].isin(l_movies_r)][['movie_Id']]

    return recomend_b


recomendar(52)


print(interact(recomendar))


interactive(children=(Dropdown(description='user_id', options=(609, 1, 2, 3, 4, 5, 6, 7, 8, 586, 585, 584, 583…

<function recomendar at 0x7ac6889b8e00>


#SISTEMA DE RECOMENDACION BASADO EN FILTROS COLABORATIVOS

In [ ]:
#instalar lightfm para sistema de recomendación de filtro colaborativo


!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831166 sha256=09fb6387ca2309778ce7aa6a6cb1d84d1c8b5f8980590b6f82b9538188f76f36
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [ ]:
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import auc_score
from sklearn.model_selection import train_test_split
#from scipy.sparse import coo_matrix
#from sklearn.model_selection import GridSearchCV

In [ ]:
ratings = pd.read_sql('SELECT * FROM db_movies_final ', conex)

#Crear data set en formato lightfm, primero lista de usuarios unicos y items únicos
dataset_train = Dataset()
dataset_test = Dataset()

all_unique_users = ratings['user_id'].unique()
all_unique_items = ratings['movie_Id'].unique()

dataset_train.fit(users=all_unique_users, items=all_unique_items)
dataset_test.fit(users=all_unique_users, items=all_unique_items)


In [ ]:

## verificar que el data set creado tenga los mismos registros que base original
cur.execute('SELECT count(distinct user_id), count (distinct movie_Id ) FROM db_movies_final ')
print(cur.fetchall())

num_users, num_items = dataset_train.interactions_shape()
print(num_users, num_items)

[(553, 3218)]
553 3218


In [ ]:
#### separar en train y test
train_df, test_df = train_test_split(ratings, test_size=0.2, random_state=42)

### para trabajar con el paquete light FM, se deben crear datos que crean consecutivos para cada usuario e item

train_interactions_list = [(row['user_id'], row['movie_Id'], row['movie_rating']) for index, row in train_df.iterrows()]

train_interactions, train_weights = dataset_train.build_interactions(train_interactions_list)

test_interactions_list = [(row['user_id'], row['movie_Id'], row['movie_rating']) for index, row in test_df.iterrows()]
test_interactions, test_weights = dataset_test.build_interactions(test_interactions_list)

In [ ]:
print(f"Shape of Interactions Matrix: {train_interactions.shape}")
print(f"Shape of Weights Matrix: {train_weights.shape}")
print(f"Shape of Interactions Matrix: {test_interactions.shape}")
print(f"Shape of Weights Matrix: {test_weights.shape}")

Shape of Interactions Matrix: (553, 3218)
Shape of Weights Matrix: (553, 3218)
Shape of Interactions Matrix: (553, 3218)
Shape of Weights Matrix: (553, 3218)


In [ ]:
#Este código es de verificación para mirar los datos cómo se estructuran
train_int_array=train_interactions.toarray()
train_weights_array=train_weights.toarray()

test_int_array=test_interactions.toarray()
test_weights_array=test_weights.toarray()

print(train_int_array)
print(test_int_array)

print(train_weights_array)
print(test_weights_array)

[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[4. 4. 4. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [4. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 3.]
 [3. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
!pip install --quiet optuna
import optuna
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.7 MB/s eta 0:00:00


In [ ]:
############# ajuste manual del modelo
loss_model='logistic'  ## 'logistic', 'bpr', 'warp', 'warp-kos': este último solo sirve para implicitas

model=LightFM(loss=loss_model, random_state=42)
model.fit(train_interactions, epochs=20, verbose=True, sample_weight=train_weights)

### auc por usuario
train_auc = auc_score(model, train_interactions)
test_auc = auc_score(model, test_interactions)


print('AUC: train %.2f, test %.2f.' % (train_auc.mean(), test_auc.mean()))


Epoch: 100%|██████████| 20/20 [00:02<00:00,  9.77it/s]


AUC: train 0.84, test 0.81.


In [49]:
#### para afinar hiperparámetros con optuna

def objective(trial):

    # grilla de hiperparámetros
    n_components = trial.suggest_int('no_components', 10, 100)       # Sugiere un entero para el número de componentes latentes (dimensionalidad del modelo).
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1, log=True) # Sugiere un flotante (en escala logarítmica) para la tasa de aprendizaje del optimizador.
    loss = trial.suggest_categorical('loss', ['logistic', 'bpr', 'warp']) # Sugiere un valor categórico para la función de pérdida ('logistic', 'bpr' o 'warp').
    epochs = trial.suggest_int('epochs', 10, 50)                     # Sugiere un entero para el número de épocas de entrenamiento.

    print(f"  Params: n_components={n_components}, lr={learning_rate:.4f}, loss={loss},  epochs={epochs}")

    # Modelo con parámetros sugeridos
    model = LightFM(
        no_components=n_components,
        learning_rate=learning_rate,
        loss=loss,

        random_state=42
    )


    model.fit(train_interactions,
              epochs=epochs,
              verbose=False,
              sample_weight=train_weights)

    test_auc = auc_score(model, test_interactions).mean()



    return test_auc


In [51]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10) # You can adjust the number of trials

#Resultados
print(f"Best trial:")
print(f"  Value (Test AUC): {study.best_value:.4f}")
print(f"  Params: {study.best_params}")


[I 2025-05-11 23:52:31,962] A new study created in memory with name: no-name-fc3c0d5f-4df0-4bbc-b4d3-7572e3ea49ff


  Params: n_components=27, lr=0.0014, loss=warp,  epochs=36


[I 2025-05-11 23:52:39,513] Trial 0 finished with value: 0.8090776205062866 and parameters: {'no_components': 27, 'learning_rate': 0.0014343376424435686, 'loss': 'warp', 'epochs': 36}. Best is trial 0 with value: 0.8090776205062866.


  Params: n_components=79, lr=0.0599, loss=bpr,  epochs=49


[I 2025-05-11 23:52:49,441] Trial 1 finished with value: 0.8225578665733337 and parameters: {'no_components': 79, 'learning_rate': 0.05992917088307126, 'loss': 'bpr', 'epochs': 49}. Best is trial 1 with value: 0.8225578665733337.


...

  Params: n_components=62, lr=0.0350, loss=logistic,  epochs=43


[I 2025-05-11 23:53:23,249] Trial 8 finished with value: 0.8093197345733643 and parameters: {'no_components': 62, 'learning_rate': 0.03503853331249373, 'loss': 'logistic', 'epochs': 43}. Best is trial 1 with value: 0.8225578665733337.


  Params: n_components=18, lr=0.0483, loss=warp,  epochs=28


[I 2025-05-11 23:53:25,945] Trial 9 finished with value: 0.8738477826118469 and parameters: {'no_components': 18, 'learning_rate': 0.048282553359225, 'loss': 'warp', 'epochs': 28}. Best is trial 9 with value: 0.8738477826118469.


Best trial:
  Value (Test AUC): 0.8738
  Params: {'no_components': 18, 'learning_rate': 0.048282553359225, 'loss': 'warp', 'epochs': 28}


In [52]:
#Entrenar modelo con mejores hyperparametros
best_params = study.best_params


model = LightFM(
    no_components=best_params['no_components'],
    learning_rate=best_params['learning_rate'],
    loss=best_params['loss'],
    random_state=42
)


model.fit(train_interactions,
                epochs=best_params['epochs'],
                verbose=True, # Set to True to see progress
                sample_weight=train_weights)


### resultado final
train_auc = auc_score(model, train_interactions).mean()
test_auc = auc_score(model, test_interactions).mean()

print('Final Model AUC: train %.4f, test %.4f.' % (train_auc, test_auc))

Epoch: 100%|██████████| 28/28 [00:02<00:00, 11.66it/s]


Final Model AUC: train 0.9537, test 0.8738.


In [53]:
list(dataset_train.mapping()[0].items())[0:10] ### para  mapeo de primerios 10 codigos originales a codigos del modelo

[(np.int64(1), 0),
 (np.int64(2), 1),
 (np.int64(3), 2),
 (np.int64(4), 3),
 (np.int64(5), 4),
 (np.int64(6), 5),
 (np.int64(7), 6),
 (np.int64(8), 7),
 (np.int64(9), 8),
 (np.int64(10), 9)]

In [57]:
def recommendation(model, data, original_user_id, conex,k):

  #data=dataset_train
  #original_user_id=1

  ## peliculas no leidos y códigos
  df_nr_movies=pd.read_sql(f'select * from db_movies_final where user_id<>{original_user_id}',conex)
  movieid_nr_movies=df_nr_movies['movie_Id'].values
  item_id_nr=[value for key, value in data.mapping()[2].items() if value not in movieid_nr_movies]

  uid_index=data.mapping()[0][original_user_id] ## id usuario según modelo

  scores=model.predict(uid_index, item_id_nr)
  sorted_indices = np.argsort(-scores).tolist()

  top_items = [key for key, value in data.mapping()[2].items() if value in sorted_indices[:k]]
  recommended=df_nr_movies[df_nr_movies['movie_Id'].isin(top_items)][['movie_Id', 'movie_title']]
  recommended.drop_duplicates(inplace=True)

  return recommended

In [58]:
recommendation (model, dataset_train,1,conex,10)

,movie_Id,movie_title
58,47,Seven (a.k.a. Se7en)
73,296,Pulp Fiction
95,608,Fargo
299,6,Heat
433,356,Forrest Gump
621,3793,X-Men
1354,1206,"Clockwork Orange, A"
1461,1029,Dumbo
1462,1030,Pete's Dragon
2428,673,Space Jam


In [55]:
import nbformat

# Path to your notebook
input_notebook = 'd_modelos2.ipynb'
output_notebook = 'd_modelos22.ipynb'

# Load the notebook
with open(input_notebook, 'r') as f:
    notebook = nbformat.read(f, as_version=4)

# Check if the notebook has 'metadata.widgets' and remove it
if 'widgets' in notebook.metadata:
    del notebook.metadata['widgets']  # Removes widgets metadata completely

# Alternatively, if you want to add a 'state' key inside 'widgets', do this:
# if 'widgets' in notebook.metadata:
#     notebook.metadata['widgets']['state'] = {}

# Save the modified notebook
with open(output_notebook, 'w') as f:
    nbformat.write(notebook, f)

print(f"Fixed notebook saved as {output_notebook}")

FileNotFoundError: [Errno 2] No such file or directory: 'd_modelos2.ipynb'